In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
%matplotlib inline

cm = plt.cm.RdBu
cm_bright = ListedColormap(['#FF0000', '#0000FF'])

import warnings
warnings.filterwarnings('ignore')


In [2]:
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler


# Cargo input

In [4]:
# Cargamos datos
pisos_20 = pd.read_csv("data origin/pisos_venta_2020.csv", delimiter=',') # cargamos fichero
pisos_21 = pd.read_csv("data origin/pisos_2021.csv", delimiter=',') # cargamos fichero

In [5]:
pisos_20['precio20']=pisos_20['precio']
pisos_21['precio21']=pisos_21['precio']

In [6]:
###### Esta funcion actualiza el número de instancias y features de mi modelo
def status_cols_rows (df):
    entries = df.shape[0]
    features =df.shape[1]
    print ('Nº de entradas:', entries)
    print ('Nº de features:', features)

In [7]:
status_cols_rows(pisos_20)
status_cols_rows(pisos_21)

Nº de entradas: 5520
Nº de features: 21
Nº de entradas: 4440
Nº de features: 26


### Observo los valores nulos

In [8]:
pisos_20.isnull().sum().sort_values(ascending=False)
pisos_21.isnull().sum().sort_values(ascending=False)

dist_centro                   658
long_                         658
lat_                          658
renta_media_barrio_persona      3
precio21                        0
n_habitaciones                  0
precio                          0
long                            0
lat                             0
metros                          0
planta                          0
tipo_vivienda                   0
n_banos                         0
ascensor_en_desc                0
desc                            0
ascensor_en_caract              0
trastero_en_desc                0
trastero_en_caract              0
garaje_en_desc                  0
garaje_en_caract                0
jardin_en_desc                  0
parquet_en_desc                 0
obra_nueva                      0
barrio                          0
ciudad                          0
titulo                          0
dtype: int64

#### Observo los tipos de cada variable

In [9]:
for c in pisos_20.columns:
    print('{}= {}'.format(c,pisos_20[c].dtype))

titulo= object
precio= object
metros= object
planta= object
tipo_vivienda= object
n_banos= object
n_habitaciones= object
ascensor_en_desc= int64
ascensor_en_caract= int64
trastero_en_desc= int64
trastero_en_caract= int64
garaje_en_desc= int64
garaje_en_caract= int64
jardin_en_desc= int64
parquet_en_desc= int64
obra_nueva= int64
barrio= object
ciudad= object
renta_media_barrio_persona= object
descripcion= object
precio20= object


## Transformaciones de algunas variables

#### Borro posibles instancias duplicadas

In [10]:
pisos_20=pisos_20.drop_duplicates()
pisos_21=pisos_21.drop_duplicates()

In [11]:
status_cols_rows(pisos_20)
status_cols_rows(pisos_21)

Nº de entradas: 5510
Nº de features: 21
Nº de entradas: 4413
Nº de features: 26


#### Creo un dataframe con los datos del 2020 que sigan en la inmobilidaria y del 2021 para comparar los precios. Lo usaré en R

In [30]:
pisos_20_21_comun=pd.merge(pisos_20, pisos_21, how='inner', on=('metros','garaje_en_desc','ascensor_en_desc','trastero_en_desc','jardin_en_desc','trastero_en_caract','ascensor_en_caract','garaje_en_caract','n_banos','n_habitaciones','planta','tipo_vivienda','barrio'), indicator='True')

In [31]:
status_cols_rows(pisos_20_21_comun)

Nº de entradas: 1490
Nº de features: 35


### Renombro el dataset de pisos21 

In [32]:
pisos_20_21=pisos_21

In [33]:
len(pisos_20_21)

4413

#### Posiciono la variable objetivo 'precio' a la primera posicion

In [34]:
# Esto solo sirve para mover 'precio' a la primera posición
cols = pisos_20_21.columns.tolist()
cols.insert(0, cols.pop(cols.index('precio')))
pisos_20_21 = pisos_20_21[cols]



In [35]:
len(pisos_20_21) 

4413

In [36]:
pisos_20_21.head()
pisos_20_21=pisos_20_21.drop('precio21', axis=1)

#### Variables cuantitativas

In [37]:
numerical_cols=pisos_20_21.dtypes=='int64'

num_cols=pisos_20_21.columns[numerical_cols]
for col in num_cols:
    
    print('\n{}'.format(col))
    print(pisos_20_21[col].unique())


ascensor_en_desc
[1 0]

ascensor_en_caract
[1 0]

trastero_en_desc
[0 1]

trastero_en_caract
[0 1]

garaje_en_desc
[0 1]

garaje_en_caract
[0 1]

jardin_en_desc
[0 1]

parquet_en_desc
[1 0]

obra_nueva
[0 1]


#### Variables cualitativas

In [38]:
categorical_cols=pisos_20_21.dtypes==object
cat_cols=pisos_20_21.columns[categorical_cols]
for col in cat_cols:
    
    print('\n{}'.format(col))
    print(pisos_20_21[col].unique())


precio
['114.900 ' '142.000 ' '58.500 ' '95.000 ' '335.000 ' '110.000 '
 '220.000 ' '260.000 ' '560.000 ' '239.000 ' '390.000 ' '209.900 '
 '215.000 ' '520.000 ' '239.900 ' '269.000 ' '130.000 ' '358.000 '
 '169.000 ' '240.000 ' '180.000 ' '177.900 ' '135.000 ' '165.000 '
 '172.000 ' '179.000 ' '212.000 ' '307.000 ' '174.000 ' '160.000 '
 '214.000 ' '151.000 ' '199.000 ' '190.000 ' '193.000 ' '188.000 '
 '256.000 ' '253.000 ' '259.500 ' '125.000 ' '387.000 ' '176.000 '
 '245.000 ' '89.000 ' '239.452 ' '133.000 ' '163.900 ' '219.000 '
 '75.600 ' '82.000 ' '198.000 ' '225.000 ' '65.000 ' '189.000 ' '181.000 '
 '199.900 ' '255.000 ' '171.535 ' '175.000 ' '99.900 ' '168.000 '
 '63.000 ' '336.000 ' '249.000 ' '315.000 ' '280.000 ' '294.698 '
 '105.950 ' '264.900 ' '301.000 ' '62.000 ' '75.000 ' '74.900 ' '320.000 '
 '244.500 ' '176.800 ' '225.900 ' '570.000 ' '274.200 ' '61.000 '
 '380.000 ' '548.000 ' '45.000 ' '231.900 ' '505.000 ' '113.900 '
 '340.000 ' '64.900 ' '54.400 ' '483.000 ' '6

### Transformo las variables planta,n_banos,n_habitaciones


Primero me aseguro que los tipos de vivienda concuerden con el numero de planta.
Si el tipo de vivienda es 'Chalet', el nº de planta es 'na', ya que se entiende que será una planta baja y que esta no debería influir en el precio, al contrario, que cuando hablamos de un bloque de pisos.
Si el tipo de vivienda es 'Casa', 'Finca' o 'Loft' y el nº de planta es un Bajo, Sotano, Entreplanta o Principal, entiendo que es una vivienda unifamiliar y por tanto la planta es 'na'.


In [39]:
pisos_20_21['tipo_vivienda'].unique()

array(['Piso', 'Apartamento', 'Casa', 'Ático', 'Chalet', 'Dúplex',
       'Estudio', 'Pisos', 'Finca', 'Loft'], dtype=object)

In [40]:
pisos_20_21['planta'].unique()

array([' 6', ' 3', 'No info', ' 4', ' 2', ' 8', ' 7', ' 5', ' 1', ' Bajo',
       ' Sótano', ' 11', ' 15', ' 9', ' 10', ' 19', ' Más de 20', '</li>',
       ' Entresuelo', ' 20', ' Subsótano', ' Principal', ' 14', ' 13',
       ' 12'], dtype=object)

In [41]:
pisos_20_21[['planta','tipo_vivienda']].head(10)


,planta,tipo_vivienda
0,6,Piso
1,3,Piso
2,No info,Piso
3,4,Piso
4,2,Piso
5,8,Piso
6,3,Piso
7,7,Apartamento
8,6,Piso
9,6,Piso


In [42]:
planta=[]
list=['No info',' Bajo', 'Sótano', ' Entresuelo', ' Principal']
for i in pisos_20_21.index:
    
    if pisos_20_21['tipo_vivienda'][i]=='Chalet':
        
        planta.append('na')
        
    elif pisos_20_21['tipo_vivienda'][i]=='Casa':
       
        planta.append('na')
        
    elif pisos_20_21['tipo_vivienda'][i]=='Finca':
        if pisos_20_21['planta'][i] in list:
            planta.append('na')
        else:
            planta.append(pisos_20_21['planta'][i])
    elif pisos_20_21['tipo_vivienda'][i]=='Loft':
        if pisos_20_21['planta'][i] in list:
            planta.append('na')
        else:
            planta.append(pisos_20_21['planta'][i])
    else:
        planta.append(pisos_20_21['planta'][i])       
        
    

In [43]:
len(planta)

4413

In [44]:
pisos_20_21['planta']=planta

Elimino caracteres de texto para poder pasar las tres variables a numericas

In [45]:
sub=['No info','</li>']
pisos_20_21=pisos_20_21.drop(pisos_20_21.loc[pisos_20_21['planta'].isin(sub)].index)
pisos_20_21=pisos_20_21.drop(pisos_20_21.loc[pisos_20_21['n_habitaciones'].isin(sub)].index)
pisos_20_21=pisos_20_21.drop(pisos_20_21.loc[pisos_20_21['n_banos'].isin(sub)].index)

In [46]:
status_cols_rows(pisos_20_21)

Nº de entradas: 3664
Nº de features: 25


Hago transformaciones en 'planta' para poder convertirla a cuantitativa

In [47]:
list=(' Entresuelo',' Bajo',' Principal', ' Sótano')
pisos_20_21['planta'].loc[pisos_20_21['planta'].isin(list)]='0'
pisos_20_21['planta'].loc[pisos_20_21['planta']=='na']='-1' #'-1' significa 'na'
pisos_20_21['planta']=pisos_20_21['planta'].replace(' Más de 20','20')

Paso las tres variables a numéricas

In [48]:
pisos_20_21['n_habitaciones']=pisos_20_21['n_habitaciones'].astype(int)
pisos_20_21['n_banos']=pisos_20_21['n_banos'].astype(int)
pisos_20_21['planta']=pisos_20_21['planta'].astype(int)

Borro los valores 'A consultar' de la columna 'Precio' y hago transformaciones para pasarla a tipo int. También paso 'metros' y 'renta_media_barrio_persona' a numéricas

In [49]:
pisos_20_21=pisos_20_21.drop(pisos_20_21.loc[pisos_20_21['precio']=='A consultar'].index)


In [ ]:
for i in pisos_20_21.index:
    pisos_20_21['precio'][i]=pisos_20_21['precio'][i].replace('.','').strip()
   
    pisos_20_21['metros'][i]=pisos_20_21['metros'][i].replace('.','').strip()
   

In [ ]:
pisos_20_21['renta_media_barrio_persona']=pisos_20_21['renta_media_barrio_persona'].astype(float)

In [593]:
pisos_20_21['precio']=pisos_20_21['precio'].astype(int)

pisos_20_21['metros']=pisos_20_21['metros'].astype(int)

Respecto a saber si una vivienda cuenta o no con ascensor, dispongo de las variables:
'ascensor_en_desc', que me indica si la palabra ascensor aparece en la descripción
'ascensor_en_caract', que me indica si la palabra ascensor aparece en las características de la vivienda de la web

Teniendo en cuenta que si en la descripción aparece el bigrama 'sin ascensor' o el trigrama 'no dispone de ascensor', entonces tengo que utilizar una variable que me clasifique las viviendas en función de si tienen o no ascensor según sea el contenido de las dos variables anteriores.

Para esto, genero una nueva variable 'ascensor' que me agrupará una serie de reglas:
Supongo que en la descripción aparecen los siguientes conjuntos que indican que no tiene ascensor:
Si ascensor aparece en las caracteristicas (ascensor_en_caract==1), la vivienda tiene ascensor --> 'ascensor' =1
SI ascensor no aparece las caracteristicas (ascensor_en_caract==0), tengo que observar si ascensor aparece o no en la descripción.

Si ascensor aparece en descripción ('ascensor_en_desc'==1), tengo que ver cómo aparece. 
Supongo que si aparece como:
'sin ascensor', 'no dispone de ascensor', 'no tiene ascensor',  la vivienda no tiene ascensor  --> 'ascensor' =0
Supongo que para el resto de apariciones, la vivienda tiene ascensor, aunque lo indique en las caracteristicas--> 'ascensor' =1

Si ascensor no aparece en descripción ('ascensor_en_desc'==1) (y ya sabiendo que tampoco aparece en las caracteristicas) --> 'ascensor' ==0

In [598]:
ascensor=[]
for i in pisos_20_21.index:
    
    if pisos_20_21['ascensor_en_caract'][i]==0:
           
    
        if pisos_20_21['ascensor_en_desc'][i]==1:
            if 'sin ascensor' in pisos_20_21['desc'][i].lower() or 'no tiene ascensor' in pisos_20_21['desc'][i].lower() or 'no dispone de ascensor' in pisos_20_21['desc'][i].lower():
                ascensor.append(0)
            
            else:
                ascensor.append(1)
        else:
            ascensor.append(0)
            
    if pisos_20_21['ascensor_en_caract'][i]==1:
        ascensor.append(1)
            

Hago lo mismo con las variables nuevas 'garaje' y 'trastero', para clasificar las viviendas en función de si estas palabras se encuentran o no en la descripción y/o en las caracteristicas indicadas en la web.

In [599]:
garaje=[]
for i in pisos_20_21.index:
    
    if pisos_20_21['garaje_en_caract'][i]==0:
           
    
        if pisos_20_21['garaje_en_desc'][i]==1:
            if 'sin garaje' in pisos_20_21['desc'][i].lower()  or 'no dispone de garaje' in pisos_20_21['desc'][i].lower():
                garaje.append(0)
            elif 'con garaje' in pisos_20_21['desc'][i].lower():
                garaje.append(1)
            else:
                garaje.append(1)
        else:
            garaje.append(0)
            
    if pisos_20_21['garaje_en_caract'][i]==1:
        garaje.append(1)
            

In [600]:
trastero=[]
for i in pisos_20_21.index:
    
    if pisos_20_21['trastero_en_caract'][i]==0:
           
    
        if pisos_20_21['trastero_en_desc'][i]==1:
            if 'sin trastero' in pisos_20_21['desc'][i].lower() or 'no tiene trastero' in pisos_20_21['desc'][i].lower() or 'no dispone de trastero' in pisos_20_21['desc'][i].lower():
                trastero.append(0)
            elif 'con trastero' in pisos_20_21['desc'][i].lower():
                trastero.append(1)
            else:
                trastero.append(1)
        else:
            trastero.append(0)
            
    if pisos_20_21['trastero_en_caract'][i]==1:
        trastero.append(1)
            

In [601]:
pisos_20_21['ascensor']=ascensor
pisos_20_21['garaje']=garaje
pisos_20_21['trastero']=trastero

In [602]:
pisos_20_21.drop(['ascensor_en_desc','ascensor_en_caract','garaje_en_desc','garaje_en_caract','trastero_en_desc','trastero_en_caract'],axis=1,inplace=True)

In [603]:
status_cols_rows(pisos_20_21)

Nº de entradas: 3637
Nº de features: 22


### Transformo las variables tipo_vivienda

Voy a ver cuantos pisos que tienen más de una habitacion están mal asignados, como tipo Apartamento.

In [604]:
len(pisos_20_21[(pisos_20_21['n_habitaciones'] >1 )& (pisos_20_21['tipo_vivienda'] =='Apartamento')])

552

Voy a ver cuántos pisos son 'Ático' y estan en la planta baja

In [605]:
len(pisos_20_21[(pisos_20_21['planta'] ==0 )& (pisos_20_21['tipo_vivienda'] =='Ático')])

0

In [606]:
falso_atico=pisos_20_21[(pisos_20_21['planta'] ==0  )& (pisos_20_21['tipo_vivienda'] =='Ático')]

In [607]:
list=('piso','apartamento','dúplex','duplex','casa')
vivienda=[]
count=0
for i in falso_atico.index:
    for j in range(0,len(list)):
        if list[j] in falso_atico['descripcion'][i].lower() :
            z=j
            countsi=1
            count+=countsi
            
        else:
            count+=count
    if count>0:
        vivienda.append(list[z])
    else:    
        vivienda.append('piso')
    count=0        
        

In [608]:
vivienda

[]

In [609]:
falso_atico['tipo_vivienda']=vivienda

In [610]:
pisos['tipo_vivienda'][(pisos['planta'] ==0 )& (pisos['tipo_vivienda'] =='Ático')]=falso_atico['tipo_vivienda']

In [611]:
falso_atico['tipo_vivienda']

Series([], Name: tipo_vivienda, dtype: float64)

En el siguiente bucle:
todas las instancias tipo 'Apartamento' con más de una habitacion y que no sean un Loft ('planta'=-1) serán pisos 

In [612]:
for i in pisos_20_21.index:
    if (pisos_20_21['n_habitaciones'][i]>1 ) and (pisos_20_21['tipo_vivienda'][i] =='Apartamento') and (pisos_20_21['planta'][i] !=-1) :
        pisos_20_21['tipo_vivienda'][i]='Piso'
        
    elif (pisos_20_21['n_habitaciones'][i]==1):
        if pisos_20_21['planta'][i] ==-1:
             pisos_20_21['tipo_vivienda'][i]='Loft'
        else:
            pisos_20_21['tipo_vivienda'][i]='Apartamento'
    else:
        pisos_20_21['tipo_vivienda'][i]=pisos_20_21['tipo_vivienda'][i]
        

Me aseguro que no haya apartamentos que deberian ser Loft

In [613]:
pisos_20_21[(pisos_20_21['planta'] =='Sin altura' )& (pisos_20_21['tipo_vivienda'] =='Apartamento')]

,precio,titulo,dist_centro,long,lat,long_,lat_,metros,planta,tipo_vivienda,...,jardin_en_desc,parquet_en_desc,obra_nueva,barrio,ciudad,renta_media_barrio_persona,desc,ascensor,garaje,trastero


### Outliers

PRECIO: Voy a suponer que lo coherente es que los precios no sean menores a 20K€ y la descripcion haga referencia a un trastero o garaje 'encubierto'

Hay 1 linea que parece ser outliers al tener un precio menor a 20000€, pero veo que puede ser coherente, ya que está en
un barrio obrero 

In [615]:
len(pisos_20_21[pisos_20_21['precio'] < 20000]) 


2

In [617]:
pisos_20_21[pisos_20_21['precio'] < 20000]


,precio,titulo,dist_centro,long,lat,long_,lat_,metros,planta,tipo_vivienda,...,jardin_en_desc,parquet_en_desc,obra_nueva,barrio,ciudad,renta_media_barrio_persona,desc,ascensor,garaje,trastero
2246,16800,Piso en Valladolid,0.018562,None,None,-4.728562,41.652133,45,2,Piso,...,0,0,0,Las Delicias (Valladolid Capital),Valladolid,18614.0,Piso en planta segunda izquierda sito en la pl...,1,0,0
2407,15200,Piso en Valladolid,0.018562,None,None,-4.728562,41.652133,44,2,Apartamento,...,0,0,0,Las Delicias (Valladolid Capital),Valladolid,18614.0,"Piso en la localidad de valladolid, dentro de ...",0,0,0


PRECIO: Voy a eliminar las instancias con precio mayor a 5M€ para posteriores calculos y gráficos

In [618]:
len(pisos_20_21[pisos_20_21['precio'] > 5000000])


0

In [619]:
pisos_20_21.drop(pisos_20_21[pisos_20_21['precio'] > 5000000].index, inplace=True)



METROS: Observo las instancias con menos de 20m y 30m. Veo que todas las instancias tienen una superficie menor a 20 m

In [620]:
len(pisos_20_21[pisos_20_21['metros'] <20])

67

In [621]:
len(pisos_20_21[pisos_20_21['metros'] <30])

68

In [622]:
pisos_20_21.drop(pisos_20_21[pisos_20_21['metros'] < 30].index, inplace=True)

N_HABITACIONES: observo aquellas viviendas con más de 8 habitaciones. 

In [623]:
pisos_20_21[(pisos_20_21['n_habitaciones'] > 8) & (pisos_20_21['metros']<200)]

,precio,titulo,dist_centro,long,lat,long_,lat_,metros,planta,tipo_vivienda,...,jardin_en_desc,parquet_en_desc,obra_nueva,barrio,ciudad,renta_media_barrio_persona,desc,ascensor,garaje,trastero
2823,450000,Piso en Indautxu,1.376780,None,None,-2.940538,43.260535,89,5,Piso,...,0,0,0,Basurtu (Distrito Basurtu-Zorrotza. Bilbao),Bilbao,24509.0,"Inmobiliaria Aguado A:P.I 558,vende maravillos...",1,1,1
3414,550000,Piso en Estación,11168.301478,None,None,-71.694699,-35.600057,175,2,Piso,...,0,1,0,Abando Hurtado de Amézaga (Distrito Abando. Bi...,Bilbao,35944.0,"Piso en venta en Abando, Plaza Circular. Edifi...",0,0,0
3580,550000,Piso en Abando,1.347654,None,None,-2.938783,43.263284,165,2,Piso,...,0,0,0,Abando Ensanche (Distrito Abando. Bilbao),Bilbao,35944.0,Piso exterior de 165 mts. junto Plaza Circular...,1,0,0
4141,550000,Piso en calle Hurtado de Amezaga,0.465503,None,None,-2.927906,43.260823,175,2,Piso,...,0,0,0,Abando Hurtado de Amézaga (Distrito Abando. Bi...,Bilbao,35944.0,"Superficie 175, Salón-comedor, 8 Habitaciones,...",1,0,1
4197,550000,Piso en Abando,1.347654,None,None,-2.938783,43.263284,175,2,Piso,...,1,0,0,Abando Ensanche (Distrito Abando. Bilbao),Bilbao,35944.0,IRIZAR VENDE. Vivienda para reformar de 165m2....,0,0,0


N_BANOS: Veo también si el numero de baños es mayor al numero de habitaciones en pisos menores a 100m2

In [624]:
pisos_20_21[(pisos_20_21['n_banos'] > pisos_20_21['n_habitaciones'])]

,precio,titulo,dist_centro,long,lat,long_,lat_,metros,planta,tipo_vivienda,...,jardin_en_desc,parquet_en_desc,obra_nueva,barrio,ciudad,renta_media_barrio_persona,desc,ascensor,garaje,trastero
257,278000,"Dúplex en calle de la Manzana, 4",37.780971,"-4,7282762","41,6527237",-4.316065,41.511227,79,3,Apartamento,...,0,0,1,Centro (Valladolid Capital),Valladolid,32596.0,3º C. Dúplex en un inmueble de obra nueva de 7...,1,0,0
311,495000,"Casa pareada en calle de Mariano de los Cobos,...",2.509733,None,None,-4.750618,41.636605,330,-1,Casa,...,1,0,0,Parquesol (Valladolid Capital),Valladolid,28580.0,Fantástico chalet pareado en la mejor zona de ...,0,1,1
492,220000,"Piso en Avenida de Salamanca, nº 1",5.557037,"-4,734472","41,6589966",-4.780186,41.620322,79,11,Apartamento,...,0,0,0,Huerta del Rey-Arturo Eyríes (Valladolid Capital),Valladolid,24683.0,Estupendo apartamento para entrar a vivir. Cue...,1,1,0
628,178000,"Apartamento en calle de Gamazo, 14",0.668947,"-4,7264189","41,6463328",-4.726189,41.646206,190,0,Apartamento,...,0,0,0,Centro (Valladolid Capital),Valladolid,32596.0,Engel &amp; Völkers les presenta un gran inmue...,1,0,1
770,220000,Piso en Huerta del Rey-Arturo Eyríes,0.918718,None,None,-4.739054,41.654433,79,11,Apartamento,...,0,0,0,Huerta del Rey-Arturo Eyríes (Valladolid Capital),Valladolid,24683.0,¿Estás buscando un piso para independizarte o ...,1,1,0
926,290000,"Chalet en calle Marina, cerca de Calle Goyescas",3.685626,None,None,-4.741902,41.683600,333,-1,Chalet,...,0,0,0,Fuente Berrocal-La Overuela (Valladolid Capital),Valladolid,22352.0,Reines castilla y león© vende este chalet lujo...,0,1,0
1135,900000,"Chalet en Urbanización Pinar de Antequera, nº 1",NaN,"-4,76398110000002","41,5775332",NaN,NaN,365,-1,Chalet,...,0,0,0,Pinar de Antequera-Puente Duero (Valladolid Ca...,Valladolid,20471.0,VIDEO INFORMATIVO: para verlo busque en youtub...,0,1,0
2192,220000,Piso en Huerta del Rey-Arturo Eyríes,0.918718,None,None,-4.739054,41.654433,79,11,Apartamento,...,0,0,0,Huerta del Rey-Arturo Eyríes (Valladolid Capital),Valladolid,24683.0,"Apartamento en venta, valladolid. Ubicado en e...",1,1,0
2231,495000,Chalet en Parquesol,3.002688,None,None,-4.758862,41.637369,356,-1,Chalet,...,0,0,0,Parquesol (Valladolid Capital),Valladolid,28580.0,"Resuelve cualquier duda, 655 75 19 62.\r<br/>E...",0,1,1
2233,139000,Chalet en calle de la Orquídea,2.890445,None,None,-4.695446,41.644116,157,-1,Chalet,...,0,0,0,Las Flores-San Isidro-Pajarillos (Valladolid C...,Valladolid,18025.0,Adosado de 3 dormitorios con garaje..,0,1,0


Borro los outliers

In [625]:
pisos_20_21.drop(pisos_20_21[(pisos_20_21['n_banos'] > pisos_20_21['n_habitaciones']) & (pisos_20_21['metros']<100)].index, inplace=True)

Tras borrar outliers, actualizo el estado de filas

In [626]:
status_cols_rows(pisos_20_21)

Nº de entradas: 3556
Nº de features: 22


Esta es la lista de variables y tipos

In [627]:
pisos_20_21.dtypes

precio                          int32
titulo                         object
dist_centro                   float64
long                           object
lat                            object
long_                         float64
lat_                          float64
metros                          int32
planta                          int32
tipo_vivienda                  object
n_banos                         int32
n_habitaciones                  int32
jardin_en_desc                  int64
parquet_en_desc                 int64
obra_nueva                      int64
barrio                         object
ciudad                         object
renta_media_barrio_persona    float64
desc                           object
ascensor                        int64
garaje                          int64
trastero                        int64
dtype: object

Voy a comprobar el porcentaje de cada categoría por cada variable

In [628]:
for c in pisos_20_21.columns:
    print('\n')
    print ('{}'.format(c))
    print('{}'.format(pisos_20_21[c].value_counts()/len(pisos_20_21)))   
    




precio
125000    0.012092
250000    0.010405
145000    0.009280
135000    0.008999
175000    0.008718
240000    0.008436
195000    0.008436
169000    0.008155
160000    0.008155
259000    0.008155
150000    0.008155
280000    0.007593
220000    0.007312
180000    0.007312
110000    0.007312
295000    0.007030
290000    0.007030
395000    0.006749
230000    0.006749
105000    0.006749
265000    0.006749
139000    0.006468
299000    0.006468
260000    0.006468
215000    0.006468
165000    0.006468
550000    0.006468
75000     0.006187
350000    0.006187
330000    0.006187
            ...   
193900    0.000281
191000    0.000281
107900    0.000281
38000     0.000281
112000    0.000281
346600    0.000281
230800    0.000281
46400     0.000281
542000    0.000281
174900    0.000281
374000    0.000281
133900    0.000281
107500    0.000281
154831    0.000281
118600    0.000281
79800     0.000281
139999    0.000281
814400    0.000281
369900    0.000281
158400    0.000281
809000    0.000281
150

In [629]:
pisos_20_21['planta'].unique()

array([ 6,  3,  4,  2,  8,  7,  5, -1,  1,  0, 11, 15,  9, 10, 19, 20, 14,
       13, 12], dtype=int64)

Elimino las variables que apenas presentan variabilidad para el modelo

In [630]:
pisos_20_21=pisos_20_21.drop(('jardin_en_desc'),axis=1)
pisos_20_21=pisos_20_21.drop(('parquet_en_desc'),axis=1)
pisos_20_21=pisos_20_21.drop(('obra_nueva'),axis=1)

Para la variable 'tipo_vivienda' puedo agrupar las que menos porcentaje tienen en el grupo 'Loft y Finca'

In [631]:
list=('Loft','Finca')
pisos_20_21['tipo_vivienda'].loc[pisos_20_21['tipo_vivienda'].isin(list)]='Loft y Finca'
pisos_20_21['tipo_vivienda'].loc[pisos_20_21['tipo_vivienda']=='duplex']='Dúplex'

Transformamos las variables planta en categóricas para disminuir la cantidad de categorías.
Para la 'planta', cuando la clasifico 'Sin altura' es que el tipo de vivienda es un chalet o una casa o una finca.

In [632]:
pisos_20_21['planta'][(pisos_20_21['planta']>5) & (pisos_20_21['planta']<=10)]=-2 #de 6 a 10
pisos_20_21['planta'][pisos_20_21['planta']>10]=-3 #mas de 10

In [633]:
pisos_20_21['planta'][pisos_20_21['planta']==-1]='Sin altura' #Sin altura
pisos_20_21['planta'][pisos_20_21['planta']==-2]='6-10'
pisos_20_21['planta'][pisos_20_21['planta']==-3]='mas de 10'

In [634]:
pisos_20_21.to_csv('pisos_cleaned_agrupando__2021.csv', sep=';', index=False)